
# FP&A Project Notebook (Auto-generated Sample Data)

**What this notebook does**
- Generates synthetic Actuals, Budget, and Forecast data for 24 months.
- Performs variance analysis and key KPI calculations (Gross Profit, Gross Margin).
- Produces simple scenario analysis (Base / Best / Worst).
- Creates basic visualizations using `matplotlib`.
- Exports outputs to Excel and CSV for Power BI.

Generated on: 2025-11-27T06:03:37.767506 UTC

Run each cell in order. This notebook uses only standard Python libraries: `pandas`, `numpy`, `matplotlib`, and `openpyxl`.


In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)


In [ ]:

# 1) Generate synthetic data (24 months)
months = pd.date_range(end=pd.Timestamp.today().normalize(), periods=24, freq='M')
np.random.seed(42)

base_revenue = 200000
monthly_growth = 0.02
base_staff_cost = 90000
base_other_cost = 30000

rows = []
for i, m in enumerate(months):
    revenue = base_revenue * ((1 + monthly_growth) ** i) * (1 + np.random.normal(0, 0.03))
    staff_cost = base_staff_cost * (1 + 0.005 * i) * (1 + np.random.normal(0, 0.02))
    other_cost = base_other_cost * (1 + 0.003 * i) * (1 + np.random.normal(0, 0.04))
    headcount = int(50 + 0.2 * i + np.random.randint(-2, 3))
    rows.append({
        'Month': m.strftime('%Y-%m'),
        'Revenue_Actual': round(revenue, 2),
        'StaffCost_Actual': round(staff_cost, 2),
        'OtherCost_Actual': round(other_cost, 2),
        'Headcount': headcount
    })

actuals = pd.DataFrame(rows).set_index('Month')
actuals.head()


In [ ]:

# 2) Create budget for months 13-24 (conservative assumptions)
budget_months = actuals.index[12:].tolist()

budget_data = []
for i, m in enumerate(budget_months):
    idx = 12 + i
    rev = base_revenue * ((1 + monthly_growth) ** idx) * 1.01
    staff = base_staff_cost * (1 + 0.005 * idx)
    other = base_other_cost * (1 + 0.003 * idx)
    budget_data.append({
        'Month': m,
        'Revenue_Budget': round(rev,2),
        'StaffCost_Budget': round(staff,2),
        'OtherCost_Budget': round(other,2)
    })

budget = pd.DataFrame(budget_data).set_index('Month')
budget.head()


In [ ]:

# 3) Simple forecast: historical (first 12) are treated as actuals; forecast for future months using avg month-over-month growth
historical = actuals.iloc[:12]
future_index = actuals.index[12:]

moms = historical['Revenue_Actual'].pct_change().dropna()
avg_mom = moms.mean()
last_known = historical['Revenue_Actual'].iloc[-1]

forecasts = []
val = last_known
for _ in future_index:
    val = val * (1 + avg_mom)
    forecasts.append(round(val,2))

forecast = pd.DataFrame({'Month': future_index, 'Revenue_Forecast': forecasts}).set_index('Month')

# Combine forecast for all months (historical -> actuals, future -> forecast)
combined_forecast = pd.concat([
    pd.DataFrame({'Revenue_Forecast': historical['Revenue_Actual']}),
    forecast
]).reindex(actuals.index)

combined_forecast.head()


In [ ]:

# 4) Merge datasets and compute variance KPIs
merged = actuals.join(budget, how='left').join(combined_forecast, how='left')

# Revenue variances
merged['Revenue_Variance_ABS'] = merged['Revenue_Actual'] - merged['Revenue_Budget']
merged['Revenue_Variance_PCT'] = merged['Revenue_Variance_ABS'] / merged['Revenue_Budget']

# Cost aggregations & variances
merged['TotalCost_Actual'] = merged['StaffCost_Actual'] + merged['OtherCost_Actual']
merged['TotalCost_Budget'] = merged['StaffCost_Budget'] + merged['OtherCost_Budget']
merged['TotalCost_Variance_ABS'] = merged['TotalCost_Actual'] - merged['TotalCost_Budget']
merged['TotalCost_Variance_PCT'] = merged['TotalCost_Variance_ABS'] / merged['TotalCost_Budget']

# Gross profit & margins
merged['GrossProfit_Actual'] = merged['Revenue_Actual'] - merged['TotalCost_Actual']
merged['GrossProfit_Budget'] = merged['Revenue_Budget'] - merged['TotalCost_Budget']
merged['GrossMargin_Actual_PCT'] = merged['GrossProfit_Actual'] / merged['Revenue_Actual']
merged['GrossMargin_Budget_PCT'] = merged['GrossProfit_Budget'] / merged['Revenue_Budget']

merged.round(2).head()


In [ ]:

# 5) Management summary (last 6 months preview + simple metrics)
summary_recent = merged.tail(6)[[
    'Revenue_Actual','Revenue_Forecast','Revenue_Budget','Revenue_Variance_ABS','Revenue_Variance_PCT',
    'TotalCost_Actual','TotalCost_Budget','TotalCost_Variance_ABS','TotalCost_Variance_PCT',
    'GrossProfit_Actual','GrossProfit_Budget','GrossMargin_Actual_PCT'
]].round(2)

summary_recent


In [ ]:

# 6) Simple scenario analysis: Best (+5% revenue), Base (as forecast), Worst (-5% revenue) for forecast period
scenario_df = merged.copy()

# Apply scenarios only to forecast months (where budget exists)
forecast_months = scenario_df.index[12:]

scenario_df['Revenue_Best'] = scenario_df['Revenue_Forecast']
scenario_df['Revenue_Worst'] = scenario_df['Revenue_Forecast']
scenario_df.loc[forecast_months, 'Revenue_Best'] = scenario_df.loc[forecast_months, 'Revenue_Forecast'] * 1.05
scenario_df.loc[forecast_months, 'Revenue_Worst'] = scenario_df.loc[forecast_months, 'Revenue_Forecast'] * 0.95

# Compute simple gross profit for scenarios (using actual costs as proxy)
scenario_df['GrossProfit_Best'] = scenario_df['Revenue_Best'] - scenario_df['TotalCost_Actual']
scenario_df['GrossProfit_Worst'] = scenario_df['Revenue_Worst'] - scenario_df['TotalCost_Actual']
scenario_df[['Revenue_Forecast','Revenue_Best','Revenue_Worst','GrossProfit_Best','GrossProfit_Worst']].head(8)


In [ ]:

# 7) Visualizations (matplotlib). Run in a notebook to see charts.

# Revenue actual vs budget vs forecast (last 12 months)
plot_df = merged[['Revenue_Actual','Revenue_Budget','Revenue_Forecast']].tail(12)
ax = plot_df.plot(marker='o', figsize=(10,5))
ax.set_title('Revenue: Actual vs Budget vs Forecast (Last 12 months)')
ax.set_ylabel('Amount')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gross margin trend (last 12 months)
gm = merged['GrossMargin_Actual_PCT'].tail(12)
ax = gm.plot(marker='s', figsize=(10,4))
ax.set_title('Gross Margin (Actual) - Last 12 months')
ax.set_ylabel('Gross Margin %')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

# 8) Export to Excel and CSV for Power BI
out_dir = Path('fpna_notebook_outputs')
out_dir.mkdir(exist_ok=True)

excel_path = out_dir / 'fpna_notebook_outputs.xlsx'
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    actuals.to_excel(writer, sheet_name='Actuals')
    budget.to_excel(writer, sheet_name='Budget')
    combined_forecast.to_excel(writer, sheet_name='Forecast')
    merged.to_excel(writer, sheet_name='Merged_Variance')
    summary_recent.to_excel(writer, sheet_name='Mgmt_Summary')

# CSVs
actuals.to_csv(out_dir / 'actuals.csv')
budget.to_csv(out_dir / 'budget.csv')
combined_forecast.to_csv(out_dir / 'forecast.csv')
merged.to_csv(out_dir / 'merged_variance.csv')

print('Exported files to', out_dir.resolve())



---

## Next steps you can do with this notebook
- Replace synthetic CSVs with your real data and adjust column mappings.
- Add more advanced forecasting (e.g., ARIMA, Prophet) for revenue and costs.
- Create a Power BI report using the exported CSVs; use `merged_variance.csv` as the main dataset.
- Expand scenario analysis with cost adjustments and headcount planning.

If you want, I can:
- add ARIMA forecasting cells,
- prepare a Power BI template,
- or produce a one-page executive PPT automatically from these results.
